# Omega computation helpers

Utilities for computing vector resonant relaxation precession rates using the `vrr_Omegas` module.

In [4]:
import numpy as np
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from typing import Any, Dict, Iterable, Mapping, Optional, Sequence, Union

import sys
from pathlib import Path

repo_root = Path.cwd().resolve()
if not (repo_root / "vrr_Omegas.py").exists():
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from vrr_Omegas import (
    Orbit,
    OrbitPair,
    ExactSeriesEvaluator,
    AsymptoticEvaluator,
    AsymptoticWithCorrectionsEvaluator,
)

sys.path.insert(1, '/fai/taras/aphi/phi-GRAPE')  # 
import phiGtools as phiG

In [5]:
_DEFAULT_METHOD_ORDER = ("exact", "asymptotic", "hybrid")


def _normalize_methods(methods: Any) -> Sequence[str]:
    """Return a tuple of unique method names derived from ``methods``."""

    if methods is None:
        return _DEFAULT_METHOD_ORDER[:1]

    if isinstance(methods, str):
        method_list = [methods]
    else:
        method_list = list(methods)

    normalized = []
    for name in method_list:
        key = str(name).strip().lower()
        if key == "all":
            return _DEFAULT_METHOD_ORDER
        if key in _DEFAULT_METHOD_ORDER and key not in normalized:
            normalized.append(key)
    if not normalized:
        raise ValueError("No valid Omega evaluation methods were provided.")
    return tuple(normalized)


def _extract_vector(row: Mapping[str, Any]) -> Optional[np.ndarray]:
    """Return the angular-momentum vector from ``row`` when available."""

    components = []
    for axis in ("Lx", "Ly", "Lz"):
        if axis not in row:
            return None
        value = row.get(axis)
        if value is None:
            return None
        try:
            value_float = float(value)
        except (TypeError, ValueError):
            return None
        if not np.isfinite(value_float):
            return None
        components.append(value_float)
    return np.asarray(components, dtype=float)


def _build_orbit(row: Mapping[str, Any], G: float, default_central_mass: float) -> Orbit:
    """Construct an :class:`Orbit` instance from a row of orbital data."""

    kwargs: Dict[str, Any] = {
        "a": float(row["a"]),
        "e": float(row["e"]),
        "m": float(row["m"]),
        "G": float(row.get("G", G)),
        "M_central": float(row.get("M_central", default_central_mass)),
    }
    vector = _extract_vector(row)
    if vector is not None:
        kwargs.update({"Lx": vector[0], "Ly": vector[1], "Lz": vector[2]})
    return Orbit(**kwargs)


def _build_evaluators(
    methods: Sequence[str],
    max_ell: int,
    method_options: Optional[Mapping[str, Mapping[str, Any]]] = None,
) -> Dict[str, Any]:
    """Instantiate evaluators for the requested methods."""

    options = {name: dict(value) for name, value in (method_options or {}).items()}
    evaluators: Dict[str, Any] = {}
    for method in methods:
        if method == "exact":
            config = options.get(method, {})
            ell_max = int(config.pop("ell_max", max_ell))
            ell_max = max(2, ell_max)
            evaluators[method] = ExactSeriesEvaluator(ell_max=ell_max, **config)
        elif method == "asymptotic":
            config = options.get(method, {})
            ell_max = int(config.pop("ell_max", max_ell))
            ell_max = max(2, ell_max)
            evaluators[method] = AsymptoticEvaluator(ell_max=ell_max, **config)
        elif method == "hybrid":
            config = options.get(method, {})
            ell_max = int(config.pop("ell_max", max_ell))
            ell_max = max(2, ell_max)
            lmax_correction = int(config.pop("lmax_correction", min(ell_max, 4)))
            lmax_correction = max(2, lmax_correction)
            evaluators[method] = AsymptoticWithCorrectionsEvaluator(
                ell_max=ell_max, lmax_correction=lmax_correction, **config
            )
        else:
            raise ValueError(f"Unsupported evaluation method: {method}")
    return evaluators


def compute_omega_for_star(args: tuple[Any, ...]) -> Dict[str, Any]:
    """Compute Omega vectors for a single star against components."""

    (
        star_i,
        stars_j_components,
        G,
        max_ell,
        component_names,
        methods,
        method_options,
    ) = args

    result: Dict[str, Any] = {"ind": star_i["ind"]}
    methods = _normalize_methods(methods)
    evaluators = _build_evaluators(methods, max_ell, method_options)

    orbit_i = _build_orbit(star_i, G, float(star_i.get("M_central", 1.0)))
    L_i_vec = orbit_i.angular_momentum_vector
    if L_i_vec is None:
        zeros = np.zeros(3, dtype=float)
        for comp in component_names:
            for method in methods:
                for axis, value in zip(("x", "y", "z"), zeros):
                    result[f"Omega_{comp}_{method}_{axis}"] = value
        return result

    norm_L_i = float(np.linalg.norm(L_i_vec))
    if norm_L_i <= 0.0:
        zeros = np.zeros(3, dtype=float)
        for comp in component_names:
            for method in methods:
                for axis, value in zip(("x", "y", "z"), zeros):
                    result[f"Omega_{comp}_{method}_{axis}"] = value
        return result

    default_central_mass = float(star_i.get("M_central", 1.0))

    for stars_j, comp_name in zip(stars_j_components, component_names):
        component_totals = {method: np.zeros(3, dtype=float) for method in methods}

        for _, star_j in stars_j.iterrows():
            if star_i["ind"] == star_j.get("ind", object()):
                continue

            orbit_j = _build_orbit(star_j, G, default_central_mass)
            L_j_vec = orbit_j.angular_momentum_vector
            if L_j_vec is None:
                continue

            norm_L_j = float(np.linalg.norm(L_j_vec))
            if norm_L_j <= 0.0:
                continue

            cos_theta = float(np.dot(L_i_vec, L_j_vec) / (norm_L_i * norm_L_j))
            cos_theta = float(np.clip(cos_theta, -1.0, 1.0))

            pair = OrbitPair(orbit_i, orbit_j, cos_theta)

            for method, evaluator in evaluators.items():
                interaction = evaluator.evaluate_pair(pair)
                omega_val = interaction.omega
                omega_vec = (
                    np.asarray(omega_val, dtype=float)
                    if isinstance(omega_val, np.ndarray)
                    else np.asarray(pair.omega_from_scalar(float(omega_val)), dtype=float)
                )
                component_totals[method] += omega_vec

        for method, vector in component_totals.items():
            for axis, value in zip(("x", "y", "z"), vector):
                result[f"Omega_{comp_name}_{method}_{axis}"] = float(value)

    return result


def compute_Omega_parallel(
    stars_i: pd.DataFrame,
    stars_j: Union[pd.DataFrame, Sequence[pd.DataFrame]],
    *,
    G: float = 1.0,
    max_ell: int = 100,
    components: Optional[Sequence[str]] = None,
    methods: Any = None,
    method_options: Optional[Mapping[str, Mapping[str, Any]]] = None,
) -> pd.DataFrame:
    """Compute Omega vectors in parallel for each star in ``stars_i``."""

    stars_i_copy = stars_i.copy()
    if "ind" not in stars_i_copy.columns:
        stars_i_copy["ind"] = range(len(stars_i_copy))

    if isinstance(stars_j, pd.DataFrame):
        stars_j_list = [stars_j.copy()]
        if components is None:
            component_names = ["comp_0"]
        else:
            component_names = list(components)
    else:
        stars_j_list = [df.copy() for df in stars_j]
        if components is None:
            component_names = [f"comp_{i}" for i in range(len(stars_j_list))]
        else:
            component_names = list(components)

    if len(component_names) != len(stars_j_list):
        raise ValueError("Number of component names must match stars_j entries.")

    for df in stars_j_list:
        if "ind" not in df.columns:
            df["ind"] = range(len(df))

    methods = _normalize_methods(methods)

    args_iterable: Iterable[tuple[Any, ...]] = [
        (
            star_i,
            stars_j_list,
            G,
            max_ell,
            component_names,
            methods,
            method_options,
        )
        for star_i in stars_i_copy.to_dict("records")
    ]

    with ProcessPoolExecutor() as executor:
        results = list(executor.map(compute_omega_for_star, args_iterable))

    omega_df = pd.DataFrame(results)
    merged = pd.merge(stars_i_copy, omega_df, on="ind", how="left")
    merged.set_index("ind", inplace=True)
    return merged


In [6]:
from scipy.spatial.transform import Rotation as R

def rotate_about_total_L(df, angle_deg=180.0):
    """
    Rotate all positions, velocities, and angular momenta in a dataframe
    by a given angle (default 180°) around the system's total angular-momentum vector.

    Parameters
    ----------
    df : pandas.DataFrame
        Must contain columns: ['x','y','z','vx','vy','vz','Lx','Ly','Lz'].
    angle_deg : float, optional
        Rotation angle in degrees (default 180).

    Returns
    -------
    df_rot : pandas.DataFrame
        Copy of the input dataframe with updated 'x','y','z','vx','vy','vz','Lx','Ly','Lz'.
        All other columns remain unchanged.
    """
    # work on a copy to avoid altering the original
    df_rot = df.copy()

    # total angular momentum vector and unit direction
    L_total = np.array([df['Lx'].sum(), df['Ly'].sum(), df['Lz'].sum()])
    L_norm = np.linalg.norm(L_total)
    if L_norm == 0:
        raise ValueError("Total angular-momentum vector is zero — cannot define rotation axis.")
    L_unit = L_total / L_norm

    # rotation object: 180° about L_unit
    rot = R.from_rotvec(np.deg2rad(angle_deg) * L_unit)

    # rotate position, velocity, and angular-momentum components
    for prefix in ['','v','L']:
        arr = df[[f'{prefix}x', f'{prefix}y', f'{prefix}z']].to_numpy()
        df_rot[[f'{prefix}x', f'{prefix}y', f'{prefix}z']] = rot.apply(arr)

    # keep identical column ordering as original
    df_rot = df_rot[df.columns]
    return df_rot



def compute_disc_disruption_condition(om_imbh_disc, om_imbh_disc_r180):
    """
    Compute the left- and right-hand sides of Eq. (fundamental criterion) from Panamarev+2025.

    Parameters
    ----------
    om_imbh_disc : pandas.DataFrame
        DataFrame containing columns:
        ['Lx','Ly','Lz','Omega_disc_x','Omega_disc_y','Omega_disc_z',
         'Omega_imbh_x','Omega_imbh_y','Omega_imbh_z']
        (and possibly other columns).
        Represents Ω_i,disc and Ω_i,IMBH(θ+Δθ).

    om_imbh_disc_r180 : pandas.DataFrame
        Same format as om_imbh_disc but rotated by 180°.
        Represents Ω_i,IMBH(θ−Δθ).

    Returns
    -------
    pandas.DataFrame
        With columns:
        ['lhs','rhs','delta_Lx','delta_Ly','delta_Lz',
         'Omega_diff_x','Omega_diff_y','Omega_diff_z']
        where lhs and rhs are |Ω_disc×L| and |((Ω_imbh(+Δθ)×L − Ω_imbh(−Δθ)×L)·ΔL̂)|
    """

    # --- Extract needed vectors ---
    L = om_imbh_disc[['Lx', 'Ly', 'Lz']].to_numpy()
    Omega_disc = om_imbh_disc[['Omega_disc_x', 'Omega_disc_y', 'Omega_disc_z']].to_numpy()
    Omega_imbh_plus = om_imbh_disc[['Omega_imbh_x', 'Omega_imbh_y', 'Omega_imbh_z']].to_numpy()
    Omega_imbh_minus = om_imbh_disc_r180[['Omega_imbh_x', 'Omega_imbh_y', 'Omega_imbh_z']].to_numpy()
    L_r180 = om_imbh_disc_r180[['Lx', 'Ly', 'Lz']].to_numpy()

    # --- ΔL̂ = (L(+Δθ) - L(−Δθ)) / |...|
    delta_L = L - L_r180
    delta_L_norm = np.linalg.norm(delta_L, axis=1, keepdims=True)
    # avoid division by zero
    delta_L_hat = np.divide(delta_L, delta_L_norm, out=np.zeros_like(delta_L), where=delta_L_norm != 0)

    # --- Left-hand side: |Ω_disc × L|
    lhs_vec = np.cross(Omega_disc, L)
    lhs = np.linalg.norm(lhs_vec, axis=1)

    # --- Right-hand side:
    # Ω_imbh(+Δθ)×L − Ω_imbh(−Δθ)×L
    cross_plus = np.cross(Omega_imbh_plus, L)
    cross_minus = np.cross(Omega_imbh_minus, L)
    Omega_diff = cross_plus - cross_minus

    # Dot product with ΔL̂ and take absolute value
    rhs_vec = np.einsum('ij,ij->i', Omega_diff, delta_L_hat)
    rhs = np.abs(rhs_vec)

    # --- Build result DataFrame ---
    result = pd.DataFrame({
        'lhs': np.abs(lhs),
        'rhs': rhs,
        'delta_Lx': delta_L[:, 0],
        'delta_Ly': delta_L[:, 1],
        'delta_Lz': delta_L[:, 2],
        'Omega_diff_x': Omega_diff[:, 0],
        'Omega_diff_y': Omega_diff[:, 1],
        'Omega_diff_z': Omega_diff[:, 2],
    })

    return result

In [7]:
p = '/fai/taras/phi-GRAPE/IMBH/1K-Plummer/1k-stardisk-imbh_a03_e01_i45_m1-Plummer-a02-M2e5-new/'
imbh_t0 = phiG.read_one_snap_disk_new(p, 0, 1000, exclude_acc = True, only_bound=False, DataFrame=True)
imbh_t1_25 = phiG.read_one_snap_disk_new(p, 4000, 1000, exclude_acc = True, only_bound=False, DataFrame=True)
imbh_t2_5 = phiG.read_one_snap_disk_new(p, 8000, 1000, exclude_acc = True, only_bound=False, DataFrame=True)
imbh_t5 = phiG.read_one_snap_disk_new(p, 15150, 1000, exclude_acc = True, only_bound=False, DataFrame=True)

In [8]:
imbh_t0_r180 = rotate_about_total_L(imbh_t0.iloc[1::2], angle_deg=180.0)
imbh_t0_r180

,ind,m,x,y,z,vx,vy,vz,a,e,...,i,r,v,ekin,ebh,L,Lx,Ly,Lz,Lc
1,1.0,1.831750e-07,-0.001191,0.007600,0.000176,-11.260131,-1.754360,-0.224380,0.007692,0.000553,...,0.000000,0.007695,11.398188,1.189893e-05,-2.380524e-05,0.087706,-2.556518e-10,-4.129491e-10,1.605821e-08,0.087706
3,3.0,8.062500e-08,0.008050,0.002217,0.000185,-2.903922,10.546741,0.224986,0.008350,0.000553,...,0.000000,0.008351,10.941532,4.826097e-06,-9.654181e-06,0.091376,-1.172345e-10,-1.893665e-10,7.363827e-09,0.091376
5,5.0,2.310250e-07,-0.006725,-0.005323,-0.000244,6.707151,-8.460727,-0.110794,0.008582,0.000553,...,0.000000,0.008580,10.797317,1.346668e-05,-2.692643e-05,0.092639,-3.405716e-10,-5.501181e-10,2.139225e-08,0.092639
7,7.0,2.512500e-08,0.007337,0.004576,0.000158,-5.694592,9.117760,0.123405,0.008645,0.000553,...,0.516509,0.008649,10.750681,1.451938e-06,-2.905072e-06,0.092979,-2.206195e-11,-4.538982e-11,2.335554e-09,0.092979
9,9.0,2.135000e-08,0.007846,-0.003901,0.000089,4.756619,9.559415,0.265476,0.008760,0.000696,...,0.516509,0.008763,10.680745,1.217786e-06,-2.436400e-06,0.093595,-4.023980e-11,-3.545082e-11,1.997525e-09,0.093595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,991.0,9.322500e-08,0.575906,0.167888,0.031062,-0.388942,1.415548,-0.399388,0.985236,0.390319,...,16.487044,0.600682,1.521369,1.078875e-07,-1.551985e-07,0.913858,-1.035004e-08,2.031644e-08,8.208662e-08,0.992591
993,993.0,2.565000e-08,-0.306115,-0.525798,0.158802,1.373160,-0.531674,-0.004782,0.987754,0.390319,...,16.487044,0.628799,1.472504,2.780805e-08,-4.079205e-08,0.915025,2.230154e-09,5.555714e-09,2.269405e-08,0.993858
995,995.0,3.212500e-07,0.034260,0.658821,0.168173,-1.293681,-0.348144,-0.369811,0.993932,0.461248,...,16.487044,0.680810,1.389811,3.102592e-07,-4.718647e-07,0.884575,-5.946047e-08,-6.582169e-08,2.699712e-07,0.996962
997,997.0,3.268000e-07,-0.024727,-0.520156,0.139208,1.645366,0.004344,0.018940,0.997239,0.461248,...,16.487045,0.539029,1.645481,4.424231e-07,-6.062754e-07,0.886046,-3.417135e-09,7.500612e-08,2.796555e-07,0.998619


In [9]:
om_imbh_disc = compute_Omega_parallel(imbh_t0.iloc[1::2], [imbh_t0.iloc[[0]],imbh_t0.iloc[1::2]], components=['imbh', 'disc'],  max_ell=50, methods=['hybrid'])
om_imbh_disc

/fai/taras/phi-GRAPE/LBIN/vrr/vrr_Omegas.py:957: RuntimeWarning: invalid value encountered in scalar divide
  return 0.5 * (log1 / (1.0 - x) - log2 / (1.0 + x))
/fai/taras/phi-GRAPE/LBIN/vrr/vrr_Omegas.py:957: RuntimeWarning: invalid value encountered in scalar divide
  return 0.5 * (log1 / (1.0 - x) - log2 / (1.0 + x))
/fai/taras/phi-GRAPE/LBIN/vrr/vrr_Omegas.py:957: RuntimeWarning: invalid value encountered in scalar divide
  return 0.5 * (log1 / (1.0 - x) - log2 / (1.0 + x))
/fai/taras/phi-GRAPE/LBIN/vrr/vrr_Omegas.py:957: RuntimeWarning: divide by zero encountered in scalar divide
  return 0.5 * (log1 / (1.0 - x) - log2 / (1.0 + x))
/fai/taras/phi-GRAPE/LBIN/vrr/vrr_Omegas.py:957: RuntimeWarning: divide by zero encountered in scalar divide
  return 0.5 * (log1 / (1.0 - x) - log2 / (1.0 + x))
/fai/taras/phi-GRAPE/LBIN/vrr/vrr_Omegas.py:337: RuntimeWarning: invalid value encountered in multiply
  return float(omega) * secondary_vector / magnitude
/fai/taras/phi-GRAPE/LBIN/vrr/vrr_Ome

,m,x,y,z,vx,vy,vz,a,e,cosi,...,Lx,Ly,Lz,Lc,Omega_imbh_hybrid_x,Omega_imbh_hybrid_y,Omega_imbh_hybrid_z,Omega_disc_hybrid_x,Omega_disc_hybrid_y,Omega_disc_hybrid_z
ind,,,,,,,,,,,,,,,,,,,,,
1.0,1.831750e-07,0.001190,-0.007602,0.000000,11.261917,1.757244,-0.000000,0.007692,0.000553,1.000000,...,0.000000e+00,0.000000e+00,1.606555e-08,0.087706,0.050858,-0.088089,0.101717,NaN,NaN,NaN
3.0,8.062500e-08,-0.008051,-0.002219,0.000000,2.902131,-10.549633,-0.000000,0.008350,0.000553,1.000000,...,0.000000e+00,0.000000e+00,7.367194e-09,0.091376,0.048810,-0.084541,0.097620,NaN,NaN,NaN
5.0,2.310250e-07,0.006727,0.005326,0.000000,-6.706269,8.462152,0.000000,0.008582,0.000553,1.000000,...,0.000000e+00,-0.000000e+00,2.140203e-08,0.092639,0.048139,-0.083379,0.096277,NaN,NaN,NaN
7.0,2.512500e-08,-0.007338,-0.004578,-0.000076,5.693772,-9.119084,-0.020396,0.008645,0.000553,0.999959,...,-1.511590e-11,-1.466279e-11,2.336004e-09,0.092979,0.048211,-0.083503,0.096421,0.004880,-0.003281,0.709153
9.0,2.135000e-08,-0.007848,0.003899,0.000064,-4.758285,-9.562107,-0.056053,0.008760,0.000696,0.999959,...,8.440736e-12,-1.591354e-11,1.998164e-09,0.093595,0.043359,-0.075101,0.086719,0.001165,0.002453,0.850276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991.0,9.322500e-08,-0.576293,-0.168513,0.017568,0.395538,-1.404894,-0.429402,0.985236,0.390319,0.958884,...,9.046634e-09,-2.242180e-08,8.169158e-08,0.992591,0.000050,-0.000086,0.000100,-0.000002,0.000016,0.000219
993.0,2.565000e-08,0.303442,0.521480,0.177116,-1.373019,0.531902,-0.012965,0.987754,0.390319,0.958884,...,-2.589867e-09,-6.136750e-09,2.250539e-08,0.993858,0.000038,-0.000065,0.000075,0.000008,0.000017,0.000206
995.0,3.212500e-07,-0.036797,-0.662919,0.150617,1.299331,0.357270,-0.340107,0.993932,0.461248,0.958884,...,5.514341e-08,5.884843e-08,2.724858e-07,0.996962,0.000009,-0.000016,0.000019,-0.000004,-0.000002,0.000203


In [11]:
om_imbh_disc_ex = compute_Omega_parallel(imbh_t0.iloc[1::2], [imbh_t0.iloc[[0]],imbh_t0.iloc[1::2]], components=['imbh', 'disc'],  max_ell=50, methods=['exact'])
om_imbh_disc_ex

,m,x,y,z,vx,vy,vz,a,e,cosi,...,Lx,Ly,Lz,Lc,Omega_imbh_exact_x,Omega_imbh_exact_y,Omega_imbh_exact_z,Omega_disc_exact_x,Omega_disc_exact_y,Omega_disc_exact_z
ind,,,,,,,,,,,,,,,,,,,,,
1.0,1.831750e-07,0.001190,-0.007602,0.000000,11.261917,1.757244,-0.000000,0.007692,0.000553,1.000000,...,0.000000e+00,0.000000e+00,1.606555e-08,0.087706,-0.000002,0.000004,-0.000005,0.000017,0.000042,-0.009123
3.0,8.062500e-08,-0.008051,-0.002219,0.000000,2.902131,-10.549633,-0.000000,0.008350,0.000553,1.000000,...,0.000000e+00,0.000000e+00,7.367194e-09,0.091376,-0.000003,0.000005,-0.000005,0.000046,0.000116,-0.020584
5.0,2.310250e-07,0.006727,0.005326,0.000000,-6.706269,8.462152,0.000000,0.008582,0.000553,1.000000,...,0.000000e+00,-0.000000e+00,2.140203e-08,0.092639,-0.000003,0.000005,-0.000006,0.000081,0.000204,-0.033157
7.0,2.512500e-08,-0.007338,-0.004578,-0.000076,5.693772,-9.119084,-0.020396,0.008645,0.000553,0.999959,...,-1.511590e-11,-1.466279e-11,2.336004e-09,0.092979,-0.000003,0.000005,-0.000006,-0.000001,0.000137,-0.079144
9.0,2.135000e-08,-0.007848,0.003899,0.000064,-4.758285,-9.562107,-0.056053,0.008760,0.000696,0.999959,...,8.440736e-12,-1.591354e-11,1.998164e-09,0.093595,-0.000003,0.000005,-0.000006,0.000128,0.000222,-0.073732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991.0,9.322500e-08,-0.576293,-0.168513,0.017568,0.395538,-1.404894,-0.429402,0.985236,0.390319,0.958884,...,9.046634e-09,-2.242180e-08,8.169158e-08,0.992591,-0.000019,0.000032,-0.000037,-0.000002,0.000016,-0.000188
993.0,2.565000e-08,0.303442,0.521480,0.177116,-1.373019,0.531902,-0.012965,0.987754,0.390319,0.958884,...,-2.589867e-09,-6.136750e-09,2.250539e-08,0.993858,-0.000015,0.000027,-0.000031,0.000008,0.000018,-0.000203
995.0,3.212500e-07,-0.036797,-0.662919,0.150617,1.299331,0.357270,-0.340107,0.993932,0.461248,0.958884,...,5.514341e-08,5.884843e-08,2.724858e-07,0.996962,-0.000012,0.000020,-0.000023,-0.000004,-0.000002,-0.000152


In [10]:
om_imbh_disc_r180 = compute_Omega_parallel(imbh_t0_r180, [imbh_t0.iloc[[0]], imbh_t0.iloc[1::2]], components=['imbh', 'disc'],  max_ell=50, methods=['hybrid'])
om_imbh_disc_r180

/fai/taras/phi-GRAPE/LBIN/vrr/vrr_Omegas.py:1083: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  val, _ = quad(integrand, -np.pi / 2.0, np.pi / 2.0, epsabs=1.0e-9, epsrel=1.0e-9, limit=200)
/fai/taras/phi-GRAPE/LBIN/vrr/vrr_Omegas.py:1083: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  val, _ = quad(integrand, -np.pi / 2.0, np.pi / 2.0, epsabs=1.0e-9, epsrel=1.0e-9, limit=200)
/fai/taras/phi-GRAPE/LBIN/vrr/vrr_Omegas.py:1083: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  val, _ = quad(integrand, -np.pi / 2.0, np.pi / 2.0, epsabs=1.0e-9, epsrel=1.0e-9, limit=200)
/fai/taras/phi-GRAPE/LBIN/vrr/vrr_Omegas.py:1083: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  val, _ = quad(integrand, -np.pi / 2.0, np.pi / 2.0, epsabs=1.0e-9, epsrel=1.0e-9, limit=200)
/fai/taras/phi-GRAPE/LBIN/vrr/vrr_Omegas.py:1083: IntegrationWarning: The integral is probably divergent, or

,m,x,y,z,vx,vy,vz,a,e,cosi,...,Lx,Ly,Lz,Lc,Omega_imbh_hybrid_x,Omega_imbh_hybrid_y,Omega_imbh_hybrid_z,Omega_disc_hybrid_x,Omega_disc_hybrid_y,Omega_disc_hybrid_z
ind,,,,,,,,,,,,,,,,,,,,,
1.0,1.831750e-07,-0.001191,0.007600,0.000176,-11.260131,-1.754360,-0.224380,0.007692,0.000553,1.000000,...,-2.556518e-10,-4.129491e-10,1.605821e-08,0.087706,0.052513,-0.090955,0.105026,2.796870e-03,-0.008681,3.243364
3.0,8.062500e-08,0.008050,0.002217,0.000185,-2.903922,10.546741,0.224986,0.008350,0.000553,1.000000,...,-1.172345e-10,-1.893665e-10,7.363827e-09,0.091376,0.050398,-0.087292,0.100796,3.349810e-03,-0.006816,2.483894
5.0,2.310250e-07,-0.006725,-0.005323,-0.000244,6.707151,-8.460727,-0.110794,0.008582,0.000553,1.000000,...,-3.405716e-10,-5.501181e-10,2.139225e-08,0.092639,0.049705,-0.086091,0.099410,3.998843e-03,-0.005234,2.475701
7.0,2.512500e-08,0.007337,0.004576,0.000158,-5.694592,9.117760,0.123405,0.008645,0.000553,0.999959,...,-2.206195e-11,-4.538982e-11,2.335554e-09,0.092979,0.049305,-0.085400,0.098611,4.148651e-03,-0.003979,1.542170
9.0,2.135000e-08,0.007846,-0.003901,0.000089,4.756619,9.559415,0.265476,0.008760,0.000696,0.999959,...,-4.023980e-11,-3.545082e-11,1.997525e-09,0.093595,0.042957,-0.074404,0.085914,4.096005e-03,-0.003592,1.694613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991.0,9.322500e-08,0.575906,0.167888,0.031062,-0.388942,1.415548,-0.399388,0.985236,0.390319,0.958884,...,-1.035004e-08,2.031644e-08,8.208662e-08,0.992591,0.000007,-0.000013,0.000015,1.589768e-05,-0.000020,0.000185
993.0,2.565000e-08,-0.306115,-0.525798,0.158802,1.373160,-0.531674,-0.004782,0.987754,0.390319,0.958884,...,2.230154e-09,5.555714e-09,2.269405e-08,0.993858,0.000013,-0.000022,0.000025,-9.443475e-07,-0.000006,0.000253
995.0,3.212500e-07,0.034260,0.658821,0.168173,-1.293681,-0.348144,-0.369811,0.993932,0.461248,0.958884,...,-5.946047e-08,-6.582169e-08,2.699712e-07,0.996962,0.000019,-0.000032,0.000037,8.884297e-06,0.000009,0.000177
